In [1]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444325 sha256=38ad208e3fad77c0812617ba7c3f530e33caaafb6ba06bba90615b6c46867b13
  Stored in directory: /home/jovyan/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [2]:
from scapy.all import *
from scapy.contrib.opc_da import *

In [45]:
scapy_cap = rdpcap('/home/jovyan/work/230404_sergnano_muletto_1onlytag_oser_qv_ler11.pcapng')
#scapy_cap = rdpcap('/home/jovyan/work/opcda_session_complete.pcapng')
#scapy_cap = rdpcap('/home/jovyan/work/syncopcda.pcapng')
server_ip = "10.26.0.210"
#server_ip = "192.168.0.112"

In [46]:
counter = 1
for packet in scapy_cap:
    try:
        if (TCP in packet and packet[TCP].dport > 20000 and packet[TCP].sport > 20000):
            packet_bytes = bytes(packet)
            opc_header = OpcDaHeaderMessage(packet_bytes)
            if (packet[IP].src == server_ip):
                print("\n**** servermessage from", packet[IP].src, "\nheader ->", opc_header.fields, "\n" )
                print("HEADER DUMP")
                hexdump(packet_bytes)
                opc_packets = ResultList(packet_bytes)
                #print(opc_packets.fields)
                for opc in opc_packets.resultList:
                    print ("\nMESSAGE =>",opc.fields)
                    hexdump(opc.raw_packet_cache)
                    #print(dir(opc))
            else:
                print ("\n**** client message from", packet[IP].src)  
            counter = counter + 1
            if ( counter > 50 ):
                break
    except Exception as error:
        print("Error on packet", packet, "because\n\t", error.with_traceback) 


**** client message from 10.64.94.3

**** servermessage from 10.26.0.210 
header -> {'versionMajor': 0, 'versionMinor': 9, 'pduType': 15, 'pfc_flags': <Flag 9 (firstFragment+reserved)>, 'integerRepresentation': 8, 'characterRepresentation': 0, 'floatingPointRepresentation': 10, 'res': 12} 

HEADER DUMP
0000  00 09 0F 09 80 0A 00 0C 29 A8 97 24 08 00 45 00  ........)..$..E.
0010  00 9C 6C C7 40 00 80 06 00 00 0A 1A 00 D2 0A 40  ..l.@..........@
0020  5E 03 C0 5F D0 14 1E 49 10 D3 A1 25 B7 79 80 18  ^.._...I...%.y..
0030  01 FE 73 BD 00 00 01 01 08 0A 06 95 3B FF 01 91  ..s.........;...
0040  7A A5 05 00 02 03 10 00 00 00 68 00 00 00 37 85  z.........h...7.
0050  03 00 50 00 00 00 03 00 00 00 00 00 00 00 00 00  ..P.............
0060  00 00 00 00 02 00 01 00 00 00 1E 54 78 41 00 61  ...........TxA.a
0070  E1 29 CF 66 D9 01 C0 00 00 00 55 73 65 72 00 00  .).f......User..
0080  00 00 03 00 00 00 00 00 00 00 04 00 19 02 0B 00  ................
0090  00 00 04 00 00 00 BC CE 29 48 04 00 02 00